# Turn original data in to Excel

In [1]:
import os
import openpyxl
import pandas as pd
folder_str='./voyager1-impacts/hits'
output_filename = './voyager1-impacts.xlsx'
combined_filename = './voyager1-impacts-unique.xlsx'

In [2]:
# Create a new Excel workbook
workbook = openpyxl.Workbook()

# Loop through all files in the folder
count=0
header = ['PART#/MOD16.MOD60', 'YYYY-DDD', 'SCET TIME']
for filename in os.listdir(folder_str):
    # Check if the filename starts with 'v1_dust_impact_list'
    if filename.startswith('v1_dust_impacts_list'):
        count+=1
        content=[]
        # If the filename matches, open the file and do something with its contents
        with open(os.path.join(folder_str, filename), 'r') as file:
            sheet_name = filename[:]  # Remove the file extension from the sheet name, if any
            worksheet = workbook.create_sheet(sheet_name)
            worksheet.append(header)

            # Read the file contents and split them into lines
            contents = file.read()
            lines = contents.split('\n')

            # Write each line to the current sheet
            for i, line in enumerate(lines):
                line = line.strip()
                if len(line)>0 and line[0].isdigit():
                    temp = line.split()
                    del temp[2]
                    content.append(temp)
            
            for row in content:
                worksheet.append(row)

                
# Delete the empty sheet
if 'Sheet' in workbook.sheetnames:
    workbook.remove(workbook['Sheet'])
# Save the workbook to disk
workbook.save(output_filename)

In [3]:
# Read the Excel file into a dictionary of DataFrames
sheets = pd.read_excel(output_filename, sheet_name=None, header=0)
count = 0
# Loop through each sheet and print its contents
for sheet_name, sheet_data in sheets.items():
    sheet_len = len(sheet_data)
    print(f'Sheet: {sheet_name}, len = {sheet_len}')
    # print(sheet_data)
    count+=sheet_len
    
print(f'count = {count}')

Sheet: v1_dust_impacts_list_may2001, len = 34
Sheet: v1_dust_impacts_list_jun2015, len = 54
Sheet: v1_dust_impacts_list_dec2005, len = 40
Sheet: v1_dust_impacts_list_nov2017, len = 57
Sheet: v1_dust_impacts_list_sep2001, len = 35
Sheet: v1_dust_impacts_list_jun2022, len = 60
Sheet: v1_dust_impacts_list_apr2020, len = 58
Sheet: v1_dust_impacts_list_mar2011, len = 50
Sheet: v1_dust_impacts_list_jan2021, len = 59
Sheet: v1_dust_impacts_list_dec2008, len = 44
Sheet: v1_dust_impacts_list_aug2012, len = 51
Sheet: v1_dust_impacts_list_mar2023, len = 61
count = 603


In [4]:
# Combine all sheets into a single DataFrame
combined_data = pd.concat(sheets.values(), ignore_index=True)
combined_data.drop_duplicates(inplace = True)
# add new YYYY-MM-DD column
combined_data['YYYY-MM-DD'] = pd.to_datetime(combined_data['YYYY-DDD'], format='%Y-%j').dt.strftime('%Y-%m-%d')

# Print the updated DataFrame
print(combined_data)

    PART#/MOD16.MOD60  YYYY-DDD     SCET TIME  YYYY-MM-DD
0          2/27204.35  1980-060  00:13:38.204  1980-02-29
1          2/27204.57  1980-060  00:31:14.204  1980-02-29
2          2/47474.21  1982-004  16:02:18.421  1982-01-04
3          2/51317.18  1982-132  18:23:55.054  1982-05-12
4          2/52730.44  1982-179  21:08:33.822  1982-06-28
..                ...       ...           ...         ...
337        9/03774.28  2018-109  03:18:17.093  2018-04-19
386        7/33570.41  2009-020  06:41:39.788  2009-01-20
387        7/47311.11  2010-113  07:05:36.061  2010-04-23
490        7/21911.11  2007-362  15:05:40.351  2007-12-28
602        9/45289.11  2022-031  23:03:45.769  2022-01-31

[80 rows x 4 columns]


In [5]:
# Write the DataFrame to the Excel file
with pd.ExcelWriter(combined_filename) as writer:
    combined_data.to_excel(writer, sheet_name='combined_data', index=False)

# Use the modified data (more accurate sclk time obtained from gif images) to get cleaned data

In [6]:
import pandas as pd
from datetime import datetime
import os
import spacepy.pycdf as pycdf
import matplotlib.pyplot as plt
import numpy as np
import time as tm

data_folder_path = './voyager-1-pws-wf/data'
excel_str = './voyager1-impacts/voyager1-impacts-unique-modified-v2.xlsx' # more accurate version
cleaned_filename = './voyager1-impacts/voyager1-impacts-unique-cleaned.xlsx'

In [7]:
# Read the Excel file into a dictionary of DataFrames
sheets = pd.read_excel(excel_str, sheet_name='combined_data', header=0)
sheets = sheets.drop_duplicates(subset=[sheets.columns[0]])
print(sheets)
print(sheets.columns)

   PART#/MOD16.MOD60  YYYY-DDD     SCET TIME  YYYY-MM-DD
0     2/27204.35.719  1980-060  00:13:38.204  1980-02-29
1     2/27204.57.170  1980-060  00:31:14.204  1980-02-29
2     2/47474.21.145  1982-004  16:02:18.421  1982-01-04
3     2/51317.18.425  1982-132  18:23:55.054  1982-05-12
4     2/52730.44.334  1982-179  21:08:33.822  1982-06-28
..               ...       ...           ...         ...
72    8/58809.58.342  2017-124  02:54:27.387  2017-05-04
74    9/03774.28.239  2018-109  03:18:17.093  2018-04-19
76    9/31227.11.188  2020-294  05:28:07.055  2020-10-20
77    9/44163.43.223  2021-359  10:41:23.424  2021-12-25
78    9/45289.11.653  2022-031  23:03:45.769  2022-01-31

[61 rows x 4 columns]
Index(['PART#/MOD16.MOD60', 'YYYY-DDD', 'SCET TIME', 'YYYY-MM-DD'], dtype='object')


In [8]:
# file path stores all the cdf files that contains a hit KV: {'YYYY-MM-DD': [PATH1, ...]}
file_paths={}

# traverse through sclk time
for index, row in sheets.iterrows():
    # get the useful information and find the name of the subdirectory
    acc_sclk = row['PART#/MOD16.MOD60']
    date_str = row['YYYY-MM-DD']
    year = date_str[:4]
    # check if sclk time is completed
    if len(acc_sclk)>11:
        year_folder_path = os.path.join(data_folder_path, year)
        # Loop through all files in the directory
        temp = [] # temp folder to store all the paths
        for file_name in os.listdir(year_folder_path):
            # Check if the file name contains the date string and ends with '.cdf'
            if date_str in file_name and file_name.endswith('.cdf'):
                # If found, print the full path to the file
                file_path = os.path.join(year_folder_path, file_name)
                temp.append(file_path)
                
            file_paths[date_str] = temp    
        

print(len(file_paths))
print(file_paths['1980-02-29'])

59
['/Volumes/Garrys_T7/Voyager/voyager-1-pws-wf/data/1980/vg1_pws_wf_1980-02-29T00_v1.0.cdf']


In [9]:
# add a new column named 'new scet'
new_scet = []
# add a new column named 'diffs' for the difference between scet and new scet diff = new scet - scet
diffs = []
# add a new column named 'indexs' for index references
indexs = []
# add a new column named 'cdf_file_paths'
cdf_file_paths = [] 

for ind, row in sheets.iterrows():
    # loop through rows to get the date and scet time
    date_str = row['YYYY-MM-DD']
    acc_sclk = row['PART#/MOD16.MOD60'].split('/')[1].replace('.', ':')
    scet = row['SCET TIME']
    dt = datetime.strptime(f"{date_str} {scet}", '%Y-%m-%d %H:%M:%S.%f')
    
    # find the path and extract the information
    file_path_lst = file_paths[date_str]
    # init values that could not exist
    index, temp_scet, diff, cdf_file_path = 'NA', 'NA', 'NA', file_path_lst
    # loop through different cdf files until we find the closest time stamp
    for num in range(len(file_path_lst)):
        file_path = file_path_lst[num]
        cdf = pycdf.CDF(file_path)
        time = cdf['Epoch'][:]
        part = cdf['Part'][:]
        sclk = cdf['SCLK'][:].tolist()
        ### BAD PRACTICE - Convert to INT in order to compar
        sclk_int = [int(s.replace(':', '')) for s in sclk]
        waveform = cdf['Waveform'][:]
        time_offsets = cdf['timeOffsets'][:]

        # check if acc_time is in range
        sclk_head, sclk_tail = sclk_int[0], sclk_int[-1]
        ### BAD PRACTICE - Convert to INT in order to compare
        acc_sclk_int = int(acc_sclk.replace(':', ''))
        # Check if acc_time is between head and tail time
        if sclk_head <= acc_sclk_int <= sclk_tail:
            # Find the closest datetime object in the list
            closest_int = min(sclk_int, key=lambda x: abs(acc_sclk_int-x))
            # Find the index of the closest datetime object in the list
            index = sclk_int.index(closest_int)
            # Get values from index
            temp_scet = time[index]
            diff = str(temp_scet - dt)
            temp_scet = temp_scet.strftime('%H:%M:%S.%f')
            cdf_file_path = file_path
#             # find unmatched frames
#             if acc_sclk_int - sclk_int[index] != 0:
#                 print(f'date={date_str}, sclk1={sclk[index-1]}, ori_sclk2={sclk[index]}, sclk3={sclk[index+1]}')
#                 print(f'data = {acc_sclk}, index = {index}')
#                 print(f'file_path = {file_path}')
            # add new info to the list   
            indexs.append(index)
            new_scet.append(temp_scet)
            diffs.append(diff)
            cdf_file_paths.append(cdf_file_path)
            
sheets['new scet'] = new_scet
sheets['diffs'] = diffs
sheets['indexs'] = indexs
sheets['cdf_file_paths'] = cdf_file_paths
print(sheets)
with pd.ExcelWriter(cleaned_filename) as writer:
    sheets.to_excel(writer, sheet_name='Sheet1', index=False)

   PART#/MOD16.MOD60  YYYY-DDD     SCET TIME  YYYY-MM-DD         new scet  \
0     2/27204.35.719  1980-060  00:13:38.204  1980-02-29  00:14:21.283999   
1     2/27204.57.170  1980-060  00:31:14.204  1980-02-29  00:31:24.344000   
2     2/47474.21.145  1982-004  16:02:18.421  1982-01-04  16:02:27.061000   
3     2/51317.18.425  1982-132  18:23:55.054  1982-05-12  18:24:20.494000   
4     2/52730.44.334  1982-179  21:08:33.822  1982-06-28  21:08:53.802000   
..               ...       ...           ...         ...              ...   
72    8/58809.58.342  2017-124  02:54:27.387  2017-05-04  02:54:45.847000   
74    9/03774.28.239  2018-109  03:18:17.093  2018-04-19  03:18:29.411000   
76    9/31227.11.188  2020-294  05:28:07.055  2020-10-20  05:28:16.274999   
77    9/44163.43.223  2021-359  10:41:23.424  2021-12-25  10:41:34.743999   
78    9/45289.11.653  2022-031  23:03:45.769  2022-01-31  23:04:22.888999   

             diffs  indexs                                     cdf_file_pat